In [1]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd
import re

def scraper_functions(urls):
    response = requests.get(urls)
    if response.status_code == 200:
        soup = bs(response.content,'html.parser')
        pattern = r'\d+\.\d+'
        pattern1 = r'\d+'

        # Créer des listes vides pour stocker les informations
        noms_entreprises = []
        notes = []
        nombres_avis = []
        pourcentages_avis_excellents = []
        domaines = []

        # Trouver la balise qui contient toutes les entreprises
        entreprises = soup.find_all('div', {'class': 'paper_paper__1PY90 paper_outline__lwsUX card_card__lQWDv card_noPadding__D8PcU styles_wrapper__2JOo2'})

        # Parcourir les entreprises et extraire les informations pertinentes
        for entreprise in entreprises:
            noms = entreprise.find('div',attrs={'class':'styles_businessUnitMain__PuwB7'}).find('p', attrs={'class': 'styles_displayName__GOhL2'}).text
            note_Span = entreprise.find('div', attrs={'class': 'styles_businessUnitMain__PuwB7'}).find('span', attrs={'class': 'styles_trustScore__8emxJ'})
            note = note_Span.text if (note_Span)is not None else ''
            match = re.search(pattern, note)
            if match:
                note = match.group(0)
            nombre = entreprise.find('div',attrs={'class':'styles_businessUnitMain__PuwB7'}).find('p', attrs={'class': 'styles_ratingText__yQ5S7'})
            nombre = nombre.text.replace('\n','').split('|')[-1].strip() if nombre is not None else ''
            match = re.search(pattern1, nombre)
            if match:
                nombre = match.group(0)
            span = entreprise.find('div',attrs={'class':'styles_businessUnitMain__PuwB7'}).find('div',attrs={'class':'star-rating_starRating__4rrcf star-rating_responsive__C9oka'})
            pourcentage = span.find('img')['alt'] if span is not None else ''

            domaine_span_list = entreprise.find('div', attrs={'class': 'styles_wrapper___E6__ styles_categoriesLabels__FiWQ4 styles_desktop__U5iWw'}).find_all('span')
            domaine = domaine_span_list[2].text.strip() if len(domaine_span_list) > 2 else ''

            # Ajouter les informations à leurs listes respectives
            noms_entreprises.append(noms)
            notes.append(note)
            nombres_avis.append(nombre.replace(',', ''))
            pourcentages_avis_excellents.append(pourcentage)
            domaines.append(domaine)

        # Créer un DataFrame pandas avec les informations
        df = pd.DataFrame({'Nom':noms_entreprises, 'Note':notes, 'Nombre d\'avis':nombres_avis, 'Pourcentage avis excellents':pourcentages_avis_excellents, 'Domaine':domaines})
        return df
    else:
        print(f"La page {urls} n'a pas pu être récupérée (code de réponse HTTP {response.status_code})")
        return None

def get_all_pages_categorie():
    categories = ['cashback_provider','bank','atm','check_cashing_service','cryptocurrency_service','currency_exchange_service','financial_institution','money_transfer_service','money_order_service','savings_bank','trust_bank','escrow_service']
    # Nombre maximal de pages à scraper pour chaque catégorie
    max_pages_per_category = [3, 7, 1, 1, 55, 9, 3, 7, 1, 1, 1, 2]

    # Scraper toutes les pages de chaque catégorie 
    urls = []  
    for i, category in enumerate(categories):
        max_pages = max_pages_per_category[i]
        for page_number in range(max_pages):
            page_url = f"https://www.trustpilot.com/categories/{category}?page={page_number+1}"
            urls.append(page_url)
    return urls    

# Scrapping all pages 
def scraping_all_page():
    pages =  get_all_pages_categorie()
    all_data = []
    for page in pages:
        df = scraper_functions(urls=page)
        all_data.append(df)
    result = pd.concat(all_data, ignore_index=True)
    return result

# Exporter les données dans un fichier CSV
data = scraping_all_page()
data.to_csv('trustpilot_data.csv', sep=',', index=False)